# Projektuppgift 1 - Kontinuerlig balk
*Tobias Ericsson, David Neinhardt*  
*2025-03-27*  
*MTM026*  

---

## Bibliotek Introducerade  
- **Pylab**: Innehåller bibliotke som numpy och plotlib som kan vara nödvändiga i uppgiften
- **Sympy**: Är biblioteket som kommer att användas för majoriteten av alla beräkningsmetoder.
- **MTM026**: Ett special tillämpat bibliotek med nödvändiga funktioner som underlättar beräkningarnas gång

In [1]:
import pylab as pl
import sympy as sp
from mtm026 import *

...

In [ ]:
# ...